In [1]:
import json


with open("../data/speeches_lemmatized.json") as f:
    speeches = json.load(f)

# Preprocessing the corpus

In [2]:
from collections import defaultdict


def group_speeches_by_quarter_century(speeches_list):
    """Group speeches into quarter centuries and create text files.

    Args:
        speeches_list: List of dictionaries containing speech data

    Returns:
        Dictionary with quarter century ranges as keys and filenames as values

    """
    # Group speeches by quarter century
    quarter_groups = defaultdict(list)

    for speech in speeches_list:
        year = int(speech["date"][:4])  # Get year from date
        # Calculate quarter century (1800-1824, 1825-1849, etc.)
        quarter_start = year - (year % 25)
        quarter_end = quarter_start + 24
        quarter_key = f"{quarter_start}-{quarter_end}"

        quarter_groups[quarter_key].append(speech)

    # Create text files for each quarter century
    file_paths = {}
    for quarter, speeches in quarter_groups.items():
        # Sort speeches by date
        speeches.sort(key=lambda x: x["date"])

        # Create lemmatized version
        lemma_text = "\n".join(s["lemmatized"] for s in speeches)
        lemma_filename = f"../data/speeches_{quarter}_lemmatized.txt"
        with open(lemma_filename, "w", encoding="utf-8") as f:
            f.write(lemma_text)

        # Create transcript version
        transcript_text = "\n".join(s["transcript"] for s in speeches)
        transcript_filename = f"../data/speeches_{quarter}_transcript.txt"
        with open(transcript_filename, "w", encoding="utf-8") as f:
            f.write(transcript_text)

        file_paths[quarter] = {
            "lemmatized": lemma_filename,
            "transcript": transcript_filename,
            "count": len(speeches),
        }

    # Print summary
    print("Quarter Century Statistics:")
    for quarter, info in file_paths.items():
        print(f"{quarter}: {info['count']} speeches")

    return file_paths


file_paths = group_speeches_by_quarter_century(speeches)

Quarter Century Statistics:
1800-1824: 59 speeches
1900-1924: 91 speeches
1975-1999: 150 speeches
1825-1849: 90 speeches
2000-2024: 164 speeches
1950-1974: 165 speeches
1925-1949: 103 speeches
1875-1899: 99 speeches
1775-1799: 28 speeches
1850-1874: 108 speeches
2025-2049: 2 speeches


# Building language models

In [ ]:
from pathlib import Path

from chronowords.algebra.svd import SVDAlgebra


def create_embeddings_for_periods(file_paths, use_lemmatized=True):
    """Create word embeddings for each quarter-century period.

    Args:
        file_paths: Dictionary from group_speeches_by_quarter_century
        use_lemmatized: Whether to use lemmatized or transcript texts

    Returns:
        Dictionary of {period: SVDAlgebra model}

    """
    models = {}

    for period, info in file_paths.items():
        print(f"\nProcessing period {period}")

        # Choose which text version to use
        filename = info["lemmatized" if use_lemmatized else "transcript"]

        # Create generator for the corpus
        def read_corpus():
            with open(filename, encoding="utf-8") as f:
                for line in f:
                    if line.strip():  # Skip empty lines
                        yield line.strip().lower()  # Lowercase everything

        # Initialize and train model
        model = SVDAlgebra(
            n_components=100,  # Smaller dimension for historical texts
            window_size=5,
            min_word_length=3,
            cms_width=1_000_000,  # 1M width should be enough for this corpus
            cms_depth=5,
        )

        try:
            # Train model
            # print(f"Training model for {period}...")
            model.train(read_corpus())

            # Print some statistics
            # print(f"Vocabulary size: {len(model.vocabulary)}")
            # print(f"Sample words: {model.vocabulary[:10]}")

            models[period] = model
        except Exception as e:
            print(f"Error processing period {period}: {e!s}")

    return models


# Create models for both lemmatized and transcript versions
print("Creating models for lemmatized texts...")
lemma_models = create_embeddings_for_periods(file_paths, use_lemmatized=True)

print("\nCreating models for transcript texts...")
transcript_models = create_embeddings_for_periods(file_paths, use_lemmatized=False)

# Save models for later use
for period, model in lemma_models.items():
    save_path = Path(f"../models/lemmatized/{period}")
    save_path.mkdir(parents=True, exist_ok=True)
    model.save_model(save_path)

for period, model in transcript_models.items():
    save_path = Path(f"../models/transcript/{period}")
    save_path.mkdir(parents=True, exist_ok=True)
    model.save_model(save_path)

Creating models for lemmatized texts...

Processing period 1800-1824
Counting words and skipgrams...


# Semantic shift

In [3]:
from pathlib import Path


def load_and_sort_models(base_path="../models/lemmatized"):
    models = {}
    for period_path in Path(base_path).iterdir():
        if period_path.is_dir():
            period = period_path.name
            loaded_model = SVDAlgebra()
            loaded_model.load_model(period_path)
            models[period] = loaded_model

    # Sort by start year
    sorted_periods = sorted(models.keys(), key=lambda x: int(x.split("-")[0]))
    return {period: models[period] for period in sorted_periods}


models = load_and_sort_models()
print("Periods in chronological order:", list(models.keys()))

target_words = [
    "freedom",
    "democracy",
    "constitution",
    "justice",
    "government",
    "power",
    "law",
    "authority",
    "america",
    "union",
    "state",
    "nation",
]

Periods in chronological order: ['1775-1799', '1800-1824', '1825-1849', '1850-1874', '1875-1899', '1900-1924', '1925-1949', '1950-1974', '1975-1999', '2000-2024', '2025-2049']


In [4]:
from chronowords.alignment.procrustes import ProcustesAligner


def analyze_shifts(models, target_words=None):
    """Analyze semantic shifts between consecutive periods."""
    periods = sorted(models.keys())
    results = {}

    for i in range(len(periods) - 1):
        period1, period2 = periods[i], periods[i + 1]
        model1, model2 = models[period1], models[period2]

        # Align embeddings
        aligner = ProcustesAligner()
        metrics = aligner.fit(
            model1.embeddings,
            model2.embeddings,
            model1.vocabulary,
            model2.vocabulary,
        )

        print(f"\nAligned {period1} -> {period2}")
        print(f"Aligned words: {metrics.num_aligned_words}")
        print(f"Average similarity: {metrics.average_cosine_similarity:.3f}")

        # Analyze specific words
        if target_words:
            shifts = []
            for word in target_words:
                sim = aligner.get_word_similarity(
                    word, model1.embeddings, model2.embeddings
                )
                if sim is not None:
                    shifts.append((word, 1 - sim))  # Convert to distance

            # Sort by shift magnitude
            shifts.sort(key=lambda x: x[1], reverse=True)
            results[f"{period1}->{period2}"] = shifts

            print("\nTop shifted words:")
            for word, shift in shifts[:5]:
                print(f"{word}: {shift:.3f}")

    return results


# Analyze key political concepts
target_words = [
    "freedom",
    "democracy",
    "government",
    "power",
    "war",
    "peace",
    "america",
    "union",
    "state",
    "constitution",
    "rights",
    "justice",
    "law",
]

shifts = analyze_shifts(models, target_words)


Aligned 1775-1799 -> 1800-1824
Aligned words: 729
Average similarity: 0.315

Top shifted words:
freedom: 1.116
power: 0.828
state: 0.804
war: 0.760
law: 0.722

Aligned 1800-1824 -> 1825-1849
Aligned words: 795
Average similarity: 0.296

Top shifted words:
peace: 0.799
constitution: 0.764
power: 0.721
union: 0.715
state: 0.709

Aligned 1825-1849 -> 1850-1874
Aligned words: 822
Average similarity: 0.299

Top shifted words:
government: 0.876
constitution: 0.799
war: 0.772
state: 0.761
union: 0.740

Aligned 1850-1874 -> 1875-1899
Aligned words: 779
Average similarity: 0.305

Top shifted words:
freedom: 0.956
constitution: 0.859
state: 0.802
america: 0.757
government: 0.715

Aligned 1875-1899 -> 1900-1924
Aligned words: 780
Average similarity: 0.301

Top shifted words:
freedom: 1.115
government: 0.770
america: 0.751
state: 0.742
constitution: 0.723

Aligned 1900-1924 -> 1925-1949
Aligned words: 782
Average similarity: 0.304

Top shifted words:
democracy: 0.983
law: 0.898
government: 0.724


In [5]:
from pathlib import Path

import altair as alt
import pandas as pd


# Load preprocessed models
models = load_and_sort_models()
periods = sorted(models.keys(), key=lambda x: int(x.split("-")[0]))

shift_data = []
# For each period transition and its shifts
for i in range(len(periods) - 1):
    period1, period2 = periods[i], periods[i + 1]
    period_pair = f"{period1} → {period2}"

    # Get the shift data for this period transition
    model1, model2 = models[period1], models[period2]
    aligner = ProcustesAligner()
    metrics = aligner.fit(
        model1.embeddings,
        model2.embeddings,
        model1.vocabulary,
        model2.vocabulary,
    )

    # Calculate shifts for target words
    for word in target_words:
        sim = aligner.get_word_similarity(word, model1.embeddings, model2.embeddings)
        if sim is not None:
            shift_data.append(
                {
                    "period": period_pair,
                    "word": word,
                    "shift": 1 - sim,  # Convert similarity to distance
                    "year": int(period1.split("-")[0]),
                }
            )

# Create DataFrame
df = pd.DataFrame(shift_data)

timeline = (
    alt.Chart(df)
    .mark_line(point=True)
    .encode(
        x=alt.X("year:Q", title="Year"),
        y=alt.Y("shift:Q", title="Semantic Change"),
        color="word:N",
        tooltip=["word", "year", "shift"],
    )
    .properties(width=800, height=400, title="Semantic Shifts Over Time")
    .interactive()
)

# Display the visualization
timeline

alt.Chart(...)

In [6]:
df

,period,word,shift,year
0,1775-1799 → 1800-1824,freedom,1.116279,1775
1,1775-1799 → 1800-1824,government,0.640784,1775
2,1775-1799 → 1800-1824,power,0.827967,1775
3,1775-1799 → 1800-1824,war,0.759982,1775
4,1775-1799 → 1800-1824,peace,0.671803,1775
...,...,...,...,...
109,2000-2024 → 2025-2049,america,0.522717,2000
110,2000-2024 → 2025-2049,state,0.769804,2000
111,2000-2024 → 2025-2049,constitution,0.707226,2000
112,2000-2024 → 2025-2049,justice,0.708987,2000


In [7]:
shift_data = []
# Use first period as reference
base_period = periods[0]
base_model = models[base_period]

for current_period in periods[1:]:
    current_model = models[current_period]
    aligner = ProcustesAligner()
    metrics = aligner.fit(
        base_model.embeddings,
        current_model.embeddings,
        base_model.vocabulary,
        current_model.vocabulary,
    )

    for word in target_words:
        sim = aligner.get_word_similarity(
            word, base_model.embeddings, current_model.embeddings
        )
        if sim is not None:
            shift_data.append(
                {"period": current_period, "word": word, "cumulative_shift": 1 - sim}
            )

df_cumulative = pd.DataFrame(shift_data)
df_cumulative["year"] = df_cumulative["period"].apply(lambda x: int(x.split("-")[0]))

cumulative_timeline = (
    alt.Chart(df_cumulative)
    .mark_line(point=True)
    .encode(
        x=alt.X("year:Q", title="Year"),
        y=alt.Y("cumulative_shift:Q", title="Cumulative Semantic Change from 1775"),
        color="word:N",
        tooltip=["word", "year", "cumulative_shift"],
    )
    .properties(
        width=800, height=400, title="Cumulative Semantic Shifts from First Period"
    )
    .interactive()
)

cumulative_timeline

alt.Chart(...)

In [8]:
df_cumulative

,period,word,cumulative_shift,year
0,1800-1824,freedom,1.116279,1800
1,1800-1824,government,0.640784,1800
2,1800-1824,power,0.827967,1800
3,1800-1824,war,0.759982,1800
4,1800-1824,peace,0.671803,1800
...,...,...,...,...
104,2025-2049,america,0.634986,2025
105,2025-2049,state,0.632132,2025
106,2025-2049,constitution,0.696486,2025
107,2025-2049,justice,0.588670,2025


# Topic modeling with NMF

In [9]:
from chronowords.topics.nmf import TopicModel


def analyze_topics_by_period(models, n_topics=10):
    """Create topic models for each time period using existing PPMI matrices.

    Args:
        models: Dictionary of period -> model data from load_and_sort_models()
        n_topics: Number of topics to extract

    """
    topic_models = {}

    for period, model_data in models.items():
        print(f"\nAnalyzing topics for {period}")

        # Get embeddings and vocabulary
        embeddings = model_data.M_dense
        embeddings[embeddings < 0.0] = 0.0  # quick fix for negative values
        vocabulary = model_data.vocabulary

        # Create topic model on existing embeddings
        topic_model = TopicModel(n_topics=n_topics)
        topic_model.fit(embeddings, vocabulary)

        # Print topics
        print("\nTop topics:")
        topic_model.print_topics()

        topic_models[period] = topic_model

    return topic_models


# Create topic models using existing data
topic_models = analyze_topics_by_period(models)


Analyzing topics for 1775-1799


/Users/varjuzoltan/Library/Caches/pypoetry/virtualenvs/chronowords-UWSi7A_J-py3.12/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(



Top topics:

Topic 0:
  service: 0.0034
  can: 0.0032
  violence: 0.0031
  london: 0.0029
  try: 0.0026
  acknowledgment: 0.0026
  upon: 0.0026
  interrupt: 0.0025
  from: 0.0025
  competent: 0.0025

Topic 1:
  never: 0.0035
  lead: 0.0031
  proportion: 0.0030
  executive: 0.0029
  condemnation: 0.0028
  academy: 0.0028
  house: 0.0027
  moderation: 0.0027
  feel: 0.0027
  heretofore: 0.0026

Topic 2:
  invite: 0.0029
  within: 0.0027
  amity: 0.0027
  four: 0.0026
  unite: 0.0026
  preservation: 0.0025
  heretofore: 0.0024
  ill: 0.0024
  expedition: 0.0024
  anxiety: 0.0022

Topic 3:
  september: 0.0032
  forget: 0.0031
  accumulation: 0.0027
  senekas: 0.0027
  obedience: 0.0027
  contribute: 0.0026
  conducive: 0.0026
  fortitude: 0.0025
  passamaquoddy: 0.0025
  contract: 0.0024

Topic 4:
  throw: 0.0030
  eye: 0.0030
  candid: 0.0029
  advantage: 0.0029
  offender: 0.0028
  since: 0.0027
  addition: 0.0027
  local: 0.0027
  age: 0.0027
  process: 0.0027

Topic 5:
  intention: 0.

/Users/varjuzoltan/Library/Caches/pypoetry/virtualenvs/chronowords-UWSi7A_J-py3.12/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(



Top topics:

Topic 0:
  improvement: 0.0037
  demand: 0.0032
  omit: 0.0028
  aid: 0.0026
  hundred: 0.0026
  return: 0.0026
  store: 0.0025
  little: 0.0025
  annual: 0.0025
  persevere: 0.0025

Topic 1:
  execution: 0.0030
  safety: 0.0028
  appoint: 0.0028
  operation: 0.0026
  seat: 0.0026
  likewise: 0.0026
  prevail: 0.0026
  difficulty: 0.0025
  survey: 0.0025
  jurisdiction: 0.0025

Topic 2:
  high: 0.0029
  port: 0.0029
  otherwise: 0.0029
  may: 0.0028
  exertion: 0.0027
  together: 0.0027
  state: 0.0026
  like: 0.0026
  subsequent: 0.0025
  event: 0.0024

Topic 3:
  employ: 0.0035
  force: 0.0029
  suffer: 0.0029
  basis: 0.0029
  have: 0.0028
  duly: 0.0028
  mean: 0.0026
  necessary: 0.0026
  employment: 0.0025
  among: 0.0025

Topic 4:
  sustain: 0.0032
  interesting: 0.0028
  next: 0.0028
  title: 0.0027
  his: 0.0026
  toward: 0.0026
  local: 0.0025
  amount: 0.0024
  take: 0.0024
  town: 0.0024

Topic 5:
  difficult: 0.0035
  stock: 0.0033
  particular: 0.0033
  repo

...

/Users/varjuzoltan/Library/Caches/pypoetry/virtualenvs/chronowords-UWSi7A_J-py3.12/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(



Top topics:

Topic 0:
  clearly: 0.0032
  employment: 0.0032
  how: 0.0031
  march: 0.0030
  total: 0.0029
  some: 0.0029
  economic: 0.0029
  fail: 0.0028
  dukakis: 0.0028
  pursue: 0.0028

Topic 1:
  benefit: 0.0040
  would: 0.0037
  200: 0.0036
  half: 0.0035
  waste: 0.0033
  search: 0.0032
  able: 0.0032
  gulf: 0.0032
  basic: 0.0031
  wide: 0.0031

Topic 2:
  put: 0.0037
  washington: 0.0032
  january: 0.0030
  private: 0.0029
  city: 0.0028
  down: 0.0027
  aid: 0.0027
  keep: 0.0027
  independence: 0.0027
  court: 0.0026

Topic 3:
  all: 0.0035
  step: 0.0031
  accomplish: 0.0031
  affirmative: 0.0028
  salt: 0.0028
  alliance: 0.0027
  close: 0.0027
  attempt: 0.0027
  town: 0.0026
  implement: 0.0026

Topic 4:
  where: 0.0038
  lead: 0.0038
  require: 0.0035
  destroy: 0.0034
  presidency: 0.0033
  enact: 0.0032
  modern: 0.0032
  foreign: 0.0031
  people: 0.0030
  balance: 0.0030

Topic 5:
  quote: 0.0037
  americans: 0.0036
  save: 0.0035
  belief: 0.0033
  ford: 0.0031


/Users/varjuzoltan/Library/Caches/pypoetry/virtualenvs/chronowords-UWSi7A_J-py3.12/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(



Top topics:

Topic 0:
  program: 0.0031
  race: 0.0031
  think: 0.0030
  believe: 0.0030
  uniform: 0.0030
  history: 0.0029
  institution: 0.0029
  response: 0.0028
  liberty: 0.0027
  her: 0.0026

Topic 1:
  his: 0.0042
  palestinian: 0.0038
  light: 0.0032
  white: 0.0031
  guarantee: 0.0031
  solution: 0.0031
  center: 0.0030
  line: 0.0030
  role: 0.0029
  afghanistan: 0.0028

Topic 2:
  reject: 0.0035
  mark: 0.0032
  audience: 0.0030
  enough: 0.0029
  judge: 0.0029
  life: 0.0029
  tear: 0.0028
  iraqis: 0.0027
  six: 0.0027
  dollar: 0.0027

Topic 3:
  usa: 0.0046
  just: 0.0038
  dr.: 0.0034
  reward: 0.0031
  approach: 0.0030
  cooperation: 0.0030
  secretary: 0.0029
  read: 0.0029
  date: 0.0029
  oppose: 0.0029

Topic 4:
  ahead: 0.0039
  brave: 0.0034
  food: 0.0034
  man: 0.0032
  authority: 0.0032
  study: 0.0030
  order: 0.0029
  whole: 0.0029
  along: 0.0028
  tell: 0.0028

Topic 5:
  hear: 0.0038
  organization: 0.0036
  innocent: 0.0035
  imagine: 0.0034
  the: 0.0

/Users/varjuzoltan/Library/Caches/pypoetry/virtualenvs/chronowords-UWSi7A_J-py3.12/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(


In [11]:
def process_topic_evolution(topic_models):
    """Process topic models into evolution chains for visualization.

    Args:
        topic_models: Dictionary mapping periods to TopicModel instances

    Returns:
        List of dicts containing evolution data for visualization

    """
    all_data = []
    # Sort periods chronologically
    periods = sorted(topic_models.keys(), key=lambda x: int(x.split("-")[0]))

    # Initialize chains from first period
    first_period = periods[0]
    first_model = topic_models[first_period]

    # Create initial topic chains
    chains = {}
    for idx, topic in enumerate(first_model.topics):
        chain_id = f"Chain_{idx}"
        top_words = [word for word, _ in topic.words[:3]]
        topic_label = ", ".join(top_words)

        chains[chain_id] = {
            "id": chain_id,
            "current_topic": topic,
            "label": topic_label,
        }

        # Add first period data point
        all_data.append(
            {
                "year": int(first_period.split("-")[0]),
                "period": first_period,
                "chain_id": chain_id,
                "topic_label": topic_label,
                "weight": topic.words[0][1],  # Use weight of top word
                "words": topic_label,
            }
        )

    # Follow alignments through subsequent periods
    for i in range(len(periods) - 1):
        period1, period2 = periods[i], periods[i + 1]
        model1 = topic_models[period1]
        model2 = topic_models[period2]

        # Get aligned topics
        aligned_topics = model1.align_with(model2)

        # Update chains based on alignments
        new_chains = {}
        used_target_topics = set()

        for aligned in aligned_topics:
            # Find chain containing source topic
            source_chain = None
            for chain_id, chain in chains.items():
                if chain["current_topic"].id == aligned.source_topic.id:
                    source_chain = chain
                    break

            if source_chain and aligned.target_topic.id not in used_target_topics:
                chain_id = source_chain["id"]
                top_words = [word for word, _ in aligned.target_topic.words[:3]]
                topic_label = ", ".join(top_words)

                # Add data point for this period
                all_data.append(
                    {
                        "year": int(period2.split("-")[0]),
                        "period": period2,
                        "chain_id": chain_id,
                        "topic_label": source_chain[
                            "label"
                        ],  # Keep original label for continuity
                        "weight": aligned.target_topic.words[0][1],
                        "words": topic_label,  # Current words for tooltip
                    }
                )

                # Update chain
                new_chains[chain_id] = {
                    "id": chain_id,
                    "current_topic": aligned.target_topic,
                    "label": source_chain["label"],
                }
                used_target_topics.add(aligned.target_topic.id)

        chains = new_chains

    return all_data


def plot_topic_trends(topic_models):
    """Create interactive visualization of topic evolution over time.

    Args:
        topic_models: Dictionary mapping periods to TopicModel instances

    Returns:
        altair.Chart: Interactive visualization

    """
    import altair as alt
    import pandas as pd

    # Process data
    all_data = process_topic_evolution(topic_models)
    df = pd.DataFrame(all_data)

    # Create base chart
    base = alt.Chart(df).encode(
        x=alt.X(
            "year:Q",
            title="Year",
            scale=alt.Scale(domain=[df.year.min(), df.year.max()]),
        ),
        color=alt.Color(
            "topic_label:N",
            legend=alt.Legend(title="Topics", orient="bottom", columns=2),
        ),
        tooltip=[
            alt.Tooltip("year:Q", title="Year"),
            alt.Tooltip("words:N", title="Top Words"),
            alt.Tooltip("weight:Q", title="Weight", format=".3f"),
        ],
    )

    # Create line chart with points
    lines = base.mark_line(size=2).encode(
        y=alt.Y("weight:Q", title="Topic Weight"), detail="chain_id:N"
    )

    points = base.mark_circle(size=60).encode(y=alt.Y("weight:Q", title="Topic Weight"))

    # Combine charts
    chart = (
        (lines + points)
        .properties(width=800, height=400, title="Topic Evolution Over Time")
        .interactive()
    )

    # Add selection
    topic_selection = alt.selection_point(fields=["topic_label"], bind="legend")

    chart = chart.add_params(topic_selection).encode(
        opacity=alt.condition(topic_selection, alt.value(1), alt.value(0.2))
    )

    return chart


# Example usage:
topic_models = analyze_topics_by_period(models)
chart = plot_topic_trends(topic_models)
chart
# chart.save('topic_evolution.html')  # Save interactive chart


Analyzing topics for 1775-1799


/Users/varjuzoltan/Library/Caches/pypoetry/virtualenvs/chronowords-UWSi7A_J-py3.12/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(



Top topics:

Topic 0:
  kind: 0.0029
  acknowledgment: 0.0028
  notify: 0.0027
  complaint: 0.0025
  from: 0.0025
  seek: 0.0025
  credit: 0.0025
  contain: 0.0024
  humane: 0.0023
  maxim: 0.0023

Topic 1:
  lead: 0.0030
  moderation: 0.0030
  never: 0.0028
  executive: 0.0028
  house: 0.0027
  proportion: 0.0026
  insure: 0.0026
  feel: 0.0026
  heretofore: 0.0025
  inform: 0.0025

Topic 2:
  repel: 0.0039
  neutral: 0.0034
  establish: 0.0028
  itself: 0.0027
  constitutional: 0.0027
  view: 0.0027
  source: 0.0027
  propose: 0.0026
  usual: 0.0026
  indeed: 0.0025

Topic 3:
  intend: 0.0029
  facilitate: 0.0029
  extinguishment: 0.0028
  7th: 0.0027
  mischief: 0.0026
  whose: 0.0025
  consolation: 0.0025
  judgment: 0.0024
  connection: 0.0024
  mutual: 0.0024

Topic 4:
  september: 0.0032
  anxiety: 0.0032
  senekas: 0.0028
  vest: 0.0026
  relate: 0.0024
  proceed: 0.0024
  recommendation: 0.0024
  passamaquoddy: 0.0024
  conducive: 0.0024
  enable: 0.0024

Topic 5:
  offender:

/Users/varjuzoltan/Library/Caches/pypoetry/virtualenvs/chronowords-UWSi7A_J-py3.12/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(



Top topics:

Topic 0:
  have: 0.0032
  duly: 0.0032
  basis: 0.0030
  recollect: 0.0029
  rapidly: 0.0029
  provinces: 0.0027
  inland: 0.0027
  necessary: 0.0026
  employ: 0.0025
  restrain: 0.0024

Topic 1:
  operation: 0.0029
  execution: 0.0028
  wish: 0.0028
  likewise: 0.0028
  appoint: 0.0028
  jurisdiction: 0.0027
  survey: 0.0026
  justice: 0.0025
  prevail: 0.0025
  safety: 0.0025

Topic 2:
  arise: 0.0028
  high: 0.0027
  with: 0.0027
  description: 0.0026
  probable: 0.0025
  port: 0.0025
  ensue: 0.0025
  disturb: 0.0025
  visit: 0.0025
  pressure: 0.0024

Topic 3:
  come: 0.0031
  bless: 0.0029
  seat: 0.0028
  ought: 0.0028
  what: 0.0027
  make: 0.0027
  remedy: 0.0026
  turn: 0.0026
  negotiation: 0.0025
  question: 0.0025

Topic 4:
  faithful: 0.0031
  wrong: 0.0030
  friendship: 0.0029
  condition: 0.0029
  difficult: 0.0028
  import: 0.0028
  confine: 0.0028
  resource: 0.0028
  militia: 0.0027
  deficiency: 0.0027

Topic 5:
  next: 0.0036
  florida: 0.0035
  inter

...

/Users/varjuzoltan/Library/Caches/pypoetry/virtualenvs/chronowords-UWSi7A_J-py3.12/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(



Top topics:

Topic 0:
  solution: 0.0036
  africa: 0.0035
  laden: 0.0033
  process: 0.0032
  governor: 0.0031
  training: 0.0030
  discussion: 0.0029
  treat: 0.0029
  debt: 0.0029
  testing: 0.0028

Topic 1:
  try: 0.0040
  authority: 0.0038
  center: 0.0034
  brave: 0.0034
  man: 0.0033
  qaeda: 0.0032
  deserve: 0.0032
  special: 0.0032
  fine: 0.0031
  five: 0.0031

Topic 2:
  reform: 0.0042
  anywhere: 0.0032
  remember: 0.0031
  usa: 0.0031
  extend: 0.0030
  why: 0.0030
  court: 0.0029
  hard: 0.0029
  will: 0.0029
  nation: 0.0028

Topic 3:
  open: 0.0033
  pressure: 0.0032
  toward: 0.0030
  may: 0.0028
  strongly: 0.0028
  never: 0.0027
  fuel: 0.0026
  factory: 0.0026
  draw: 0.0026
  problem: 0.0026

Topic 4:
  keep: 0.0041
  entrepreneur: 0.0033
  ship: 0.0033
  announce: 0.0031
  wrong: 0.0030
  suppose: 0.0029
  lose: 0.0029
  impose: 0.0029
  senate: 0.0029
  stake: 0.0028

Topic 5:
  dollar: 0.0040
  audience: 0.0035
  trade: 0.0032
  wonder: 0.0032
  civilian: 0.003

/Users/varjuzoltan/Library/Caches/pypoetry/virtualenvs/chronowords-UWSi7A_J-py3.12/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn(



Top topics:

Topic 0:
  vice: 0.0034
  earn: 0.0034
  disposal: 0.0033
  drug: 0.0033
  pull: 0.0031
  obligation: 0.0031
  punish: 0.0031
  oligarchy: 0.0030
  now: 0.0029
  optimistic: 0.0029

Topic 1:
  threat: 0.0037
  unlimited: 0.0032
  purpose: 0.0032
  harness: 0.0031
  into: 0.0031
  about: 0.0030
  completely: 0.0029
  totally: 0.0029
  whom: 0.0029
  should: 0.0029

Topic 2:
  network: 0.0031
  tech: 0.0031
  immediately: 0.0030
  vehicle: 0.0030
  waste: 0.0030
  female: 0.0029
  back: 0.0029
  check: 0.0029
  stay: 0.0029
  far: 0.0028

Topic 3:
  sway: 0.0039
  future: 0.0038
  raise: 0.0033
  reclaim: 0.0033
  security: 0.0031
  object: 0.0031
  external: 0.0031
  transition: 0.0030
  walk: 0.0029
  office: 0.0029

Topic 4:
  life: 0.0034
  beautiful: 0.0033
  earn: 0.0033
  day: 0.0032
  meanwhile: 0.0031
  provide: 0.0030
  chief: 0.0029
  honest: 0.0028
  great: 0.0027
  gold: 0.0027

Topic 5:
  expel: 0.0041
  harbor: 0.0037
  unjustly: 0.0031
  consequential: 0.003

alt.LayerChart(...)